In [1]:
import pandas as pd

In [2]:
import re

In [3]:
from indic_transliteration import sanscript
from indic_transliteration.sanscript import transliterate

In [4]:
from nltk.tokenize import word_tokenize

In [5]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForTokenClassification

tokenizer = AutoTokenizer.from_pretrained("l3cube-pune/me-lid-bert")
model = AutoModelForTokenClassification.from_pretrained("l3cube-pune/me-lid-bert")

config.json:   0%|          | 0.00/822 [00:00<?, ?B/s]

D:\Mr\venv\Lib\site-packages\huggingface_hub\file_download.py:159: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\kunal\.cache\huggingface\hub\models--l3cube-pune--me-lid-bert. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


model.safetensors:   0%|          | 0.00/948M [00:00<?, ?B/s]

In [6]:
df = pd.read_csv('MeRoman_500.csv')

In [7]:
df

,text
0,"@USER Barfi be like - nigh lavkar, I need to s..."
1,Kakaebabe shakto buh?lol
2,tu khup ghoshti chiglun nehtes.... Tuja jaag v...
3,@USER sir tumhala asa vatat nahi ka ki Sanjay ...
4,Jevha khahi pramanat rugna hoote tevha school ...
...,...
495,@USER tumchya prarthnela nakki yash yeil yachi...
496,tumhi chalu dya tumcha tumcha. Me Anna Hazare ...
497,@USER @USER Amhala Tumhich ch Home minister mh...
498,@USER @USER @USER saheb mag tasa tax Central G...


In [8]:
X = df['text']

In [9]:
X

0      @USER Barfi be like - nigh lavkar, I need to s...
1                               Kakaebabe shakto buh?lol
2      tu khup ghoshti chiglun nehtes.... Tuja jaag v...
3      @USER sir tumhala asa vatat nahi ka ki Sanjay ...
4      Jevha khahi pramanat rugna hoote tevha school ...
                             ...                        
495    @USER tumchya prarthnela nakki yash yeil yachi...
496    tumhi chalu dya tumcha tumcha. Me Anna Hazare ...
497    @USER @USER Amhala Tumhich ch Home minister mh...
498    @USER @USER @USER saheb mag tasa tax Central G...
499    @USER tv9 news wale kuth ahe.......Apla msg sa...
Name: text, Length: 500, dtype: object

In [10]:
def process_text(text):
    tokens = re.sub(r'(?:@USER)', '', text)
    tokens = [token for token in word_tokenize(tokens) if token.isalnum()]
    tokens = " ".join(tokens)
    
    tokens = transliterate(tokens,sanscript.ITRANS,sanscript.DEVANAGARI)
    return tokens

In [11]:
# ["My", "Name", "is", "Kunal"]

In [12]:
cleaned = X.apply(process_text)

In [13]:
cleaned

0      Bअर्फ़ि बे लिके निघ् लव्कर् ई नीद् तो शिफ़्त् ओन...
1                                 ख़कएबबे शक्तो बुह् लोल्
2      तु खुप् घोश्ति चिग्लुन् नेह्तेस् टुज जाग् वेग्...
3      सिर् तुम्हल अस वतत् नहि क कि षन्जय् नवतच् मुलत...
4      ज़ेव्ह खहि प्रमनत् रुग्न हूते तेव्ह स्चूल् अम्प...
                             ...                        
495    तुम्च्य प्रर्थ्नेल नक्कि यश् येइल् यचि खत्रि अ...
496    तुम्हि चलु द्य तुम्च तुम्च ंए आन्न ःअज़रे बोल्तोय्
497    आम्हल टुम्हिच् च् ःओमे मिनिस्तेर् म्हनुन् हवेत...
498    सहेब् मग् तस तक्ष् छेन्त्रल् ग़ोवेर्न्मेन्त् ल ...
499    त्व्९ नेव्स् वले कुथ् अहे आप्ल म्स्ग् सर्कर् प...
Name: text, Length: 500, dtype: object

In [14]:
df['text']

0      @USER Barfi be like - nigh lavkar, I need to s...
1                               Kakaebabe shakto buh?lol
2      tu khup ghoshti chiglun nehtes.... Tuja jaag v...
3      @USER sir tumhala asa vatat nahi ka ki Sanjay ...
4      Jevha khahi pramanat rugna hoote tevha school ...
                             ...                        
495    @USER tumchya prarthnela nakki yash yeil yachi...
496    tumhi chalu dya tumcha tumcha. Me Anna Hazare ...
497    @USER @USER Amhala Tumhich ch Home minister mh...
498    @USER @USER @USER saheb mag tasa tax Central G...
499    @USER tv9 news wale kuth ahe.......Apla msg sa...
Name: text, Length: 500, dtype: object

In [24]:
df['text'][7]

'@USER Devendraji Police station la jaun polisanchi ijjat ghetil tyamule open ijjat Janya peksha devendrajimchya ghari char bhinti madhe apali ijjat ghalavatil police '

In [23]:
import torch

# Tokenize input text
inputs = tokenizer(df['text'][7], return_tensors="pt", padding=True, truncation=True)
token = word_tokenize(df['text'][7])

# Get model outputs
with torch.no_grad():
    outputs = model(**inputs)
logits = outputs.logits

# Convert logits to predictions
predictions = torch.argmax(logits, dim=-1)

# Map predictions to labels
labels = ["Marathi", "English", "Other"]  # Replace with your actual labels
predicted_labels = [labels[pred] for pred in predictions[0].tolist()]

for i in range(len(predicted_labels)):
    print(f'{token[i]}: {predicted_labels[i]}')

@: Other
USER: Other
Devendraji: Other
Police: Other
station: Other
la: Marathi
jaun: English
polisanchi: English
ijjat: Marathi
ghetil: Marathi
tyamule: Marathi
open: Marathi
ijjat: Marathi
Janya: Marathi
peksha: Marathi
devendrajimchya: Marathi
ghari: Marathi
char: Marathi
bhinti: Marathi
madhe: English
apali: Marathi
ijjat: Marathi
ghalavatil: Marathi
police: Marathi


IndexError: list index out of range

In [18]:
from transformers import AutoConfig

config = AutoConfig.from_pretrained("l3cube-pune/me-lid-bert")

# Print the configuration
print(config)

BertConfig {
  "_name_or_path": "l3cube-pune/me-lid-bert",
  "architectures": [
    "BertForTokenClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "embedding_size": 768,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "Marathi",
    "1": "English",
    "2": "Other"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "English": 1,
    "Marathi": 0,
    "Other": 2
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "torch_dtype": "float32",
  "transformers_version": "4.43.3",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 197285
}



In [ ]:
# Tokenize input text
